In [1]:
#import thư viện
import os
import cv2
import pickle
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [2]:
#path cho data
BASE_DIR = os.getcwd()
image_dir = os.path.join(BASE_DIR, 'data', 'input')
annot_dir = os.path.join(BASE_DIR, 'data', 'annotations')

In [3]:
#load data gồm các thông số từ file XML với ảnh tương ứng
label2category = {'without_mask': 0, 'with_mask': 1}
category2label = {v: k for k, v in label2category.items()}
datas = []

for root, dirs, files in os.walk(annot_dir):
    for file in files:
        tree = ET.parse(os.path.join(root, file))
        data = {'path': None, 'objects': []}
        data['path'] = os.path.join(image_dir, tree.find('filename').text)
        for obj in tree.findall('object'):
            label = label2category[obj.find('name').text]
            xmin = int(obj.find('bndbox/xmin').text)
            ymin = int(obj.find('bndbox/ymin').text)
            xmax = int(obj.find('bndbox/xmax').text)
            ymax = int(obj.find('bndbox/ymax').text)
            data['objects'].append([label, xmin, ymin, xmax, ymax])
        datas.append(data)

Total images : 1370


In [4]:
#lưu vào mảng X và Y, X chứa trọng số của data, Y chứa nhãn của data
img_size = (100, 100)
X = []
Y = []

for data in datas:
    img = cv2.imread(data['path'])
    for (category, xmin, ymin, xmax, ymax) in data['objects']:
        roi = img[ymin : ymax, xmin : xmax]
        roi = cv2.resize(roi, (100, 100))
        data = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        target = to_categorical(category, num_classes=len(category2label))
        X.append(data)
        Y.append(target)
        
X = np.array(X)
Y = np.array(Y)

In [5]:
np.save('data/X', X)
np.save('data/Y', Y)

In [6]:
with open('data/category2label.pkl', 'wb') as pf:
    pickle.dump(category2label, pf)